In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path

artifact_torch_root = Path().absolute().parent.parent

os.chdir(artifact_torch_root)

Path().absolute()

In [ ]:
import pandas as pd
import seaborn as sns
from artifact_core.binary_classification import BinaryFeatureSpec
from artifact_experiment import DataSplit
from artifact_experiment.tracking import FilesystemTrackingClient
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

from demos.binary_classification.config.constants import (
    EXPERIMENT_ID,
    LABEL_FEATURE,
    LS_CATEGORIES,
    LS_FEATURES,
    POSITIVE_CATEGORY,
    TRAINING_DATASET_PATH,
    VAL_DATA_PROPORTION,
)
from demos.binary_classification.data.utils import DemoDataUtils
from demos.binary_classification.experiment.experiment import DemoBinaryClassificationExperiment
from demos.binary_classification.model.classifier import MLPClassifier

In [ ]:
sns.set_theme(style="whitegrid", palette="colorblind")

In [ ]:
df_all = pd.read_csv(artifact_torch_root / TRAINING_DATASET_PATH)

df_all

In [ ]:
class_spec = BinaryFeatureSpec(
    ls_categories=LS_CATEGORIES, positive_category=POSITIVE_CATEGORY, feature_name=LABEL_FEATURE
)

class_spec

In [ ]:
df_train, df_val = train_test_split(
    df_all,
    test_size=VAL_DATA_PROPORTION,
    random_state=42,
    shuffle=True,
)

df_train

In [ ]:
data_loaders = {
    DataSplit.TRAIN: DemoDataUtils.build_data_loader(df=df_train, class_spec=class_spec),
    DataSplit.VALIDATION: DemoDataUtils.build_data_loader(df=df_val, class_spec=class_spec),
}

In [ ]:
artifact_routine_data = {
    DataSplit.TRAIN: DemoDataUtils.build_artifact_routine_data(df=df_train, class_spec=class_spec),
    DataSplit.VALIDATION: DemoDataUtils.build_artifact_routine_data(
        df=df_val, class_spec=class_spec
    ),
}

In [ ]:
model = MLPClassifier.build(class_spec=class_spec, ls_features=LS_FEATURES)

In [ ]:
tracking_client = FilesystemTrackingClient.build(experiment_id=EXPERIMENT_ID)

In [ ]:
experiment = DemoBinaryClassificationExperiment.build(
    model=model,
    data_loaders=data_loaders,
    artifact_routine_data=artifact_routine_data,
    artifact_routine_data_spec=class_spec,
    tracking_client=tracking_client,
)

In [ ]:
experiment.run()

In [ ]:
experiment.epoch_scores

In [ ]:
plt.figure(figsize=(10, 6))
experiment.epoch_scores["LOSS_EPOCH"].dropna().plot(
    color=sns.color_palette("colorblind")[2], linewidth=2, label="train"
)
experiment.epoch_scores["LOSS_VAL"].dropna().plot(
    color="red",
    linewidth=2,
    label="val",
)
plt.legend()
plt.title("Loss", fontsize=16, fontweight="bold")
plt.xlabel("Epoch", fontsize=14)
plt.ylabel("Loss", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.6)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()